In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
import numpy as np

# Pandas私房手册-基本索引类型

## `Index`索引对象

### 设置元数据

索引大多是不可变的，但是可以设置和更改它们的元数据，比如索引名称`name`(以及层级索引的`levels`和`codes`属性)。可以使用`rename`、`set_names`、`set_levels`和`set_codes`来直接设置这些属性，默认返回一个副本，但是，也可以设置`inplace=True`在原地修改，更多的内容可以参考《MultiIndex对象和层级索引》一章。

In [3]:
index = pd.MultiIndex.from_product([range(3), ['one', 'two']], names=['first', 'second'])
index.levels[0]
index.get_level_values(0)
index.set_levels(['a', 'b', 'c'], level=0)

Int64Index([0, 1, 2], dtype='int64', name='first')

Int64Index([0, 0, 1, 1, 2, 2], dtype='int64', name='first')

MultiIndex([('a', 'one'),
            ('a', 'two'),
            ('b', 'one'),
            ('b', 'two'),
            ('c', 'one'),
            ('c', 'two')],
           names=['first', 'second'])

### `Index`对象的集合操作

两个主要操作是并集`union(|)`和交集`intersection(&)`以及对称差集`symmetric_difference (^)`和差集`difference`方法。前三者可以直接作为方法调用，也可以通过重载操作符使用：

In [4]:
a = pd.Index(['c', 'b', 'a'])
b = pd.Index(['c', 'e', 'd'])
a | b
a.difference(b)
a ^ b

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

Index(['a', 'b'], dtype='object')

Index(['a', 'b', 'd', 'e'], dtype='object')

### 设置`set_index`和重设`reset_index`索引

`set_index()`方法有个`append`参数，允许保留现有索引，把指定的列追加到现有索引后面成为层级索引：

In [5]:
df = DataFrame(np.random.randn(4, 3))
df[3] = ['A', 'A', 'B', 'B']
df.set_index(3)
df.set_index(3, append=True)

,0,1,2
3,,,
A,1.158441,-0.855127,-0.159237
A,0.343511,1.808090,0.064427
B,0.231911,1.192244,0.471319
B,-0.416951,-0.626586,0.842836


,,0,1,2
,3,,,
0,A,1.158441,-0.855127,-0.159237
1,A,0.343511,1.808090,0.064427
2,B,0.231911,1.192244,0.471319
3,B,-0.416951,-0.626586,0.842836


`set_index()`有个`drop`参数，表示将列设置为索引后还是否作为列保留（默认为False）：

In [6]:
df.set_index(3, drop=False)

,0,1,2,3
3,,,,
A,1.158441,-0.855127,-0.159237,A
A,0.343511,1.808090,0.064427,A
B,0.231911,1.192244,0.471319,B
B,-0.416951,-0.626586,0.842836,B


重设索引`reset_index()`可以将索引变为列，层级索引的话，可以通过`level`参数指定重设哪一层，它也有个`drop`参数，如果为True，则会直接丢弃而不会转成列：

In [7]:
df1 = df.set_index(3, append=True)
df1
df1.reset_index(level=1, drop=True)

,,0,1,2
,3,,,
0,A,1.158441,-0.855127,-0.159237
1,A,0.343511,1.808090,0.064427
2,B,0.231911,1.192244,0.471319
3,B,-0.416951,-0.626586,0.842836


,0,1,2
0,1.158441,-0.855127,-0.159237
1,0.343511,1.808090,0.064427
2,0.231911,1.192244,0.471319
3,-0.416951,-0.626586,0.842836


## `CategoricalIndex`索引类型

### 创建`CategoricalIndex`索引类型

这里大致说明一下`Categorical`，`CategoricalDtype`，`CategoricalIndex`的区别：可以认为`CategoricalDtype`是一种类型，默认是字母输入时候的排列顺序。`Categorical`创建一个序列，需要指定序列的值以及对应的`CategoricalDtype`类型，该序列有一个`code`属性，是和类别对应的整数数列。`CategoricalIndex`是一种索引类型，其实是对`pandas`的`Categorical`又进行了一次包装，用于支持使用重复索引，简单说，就是如果索引有大量的重复的元素，那么转换成`CategoricalIndex`可以极大的减少内存，并且提高计算速度。  
仔细分辨下面几种创建`CategoricalIndex`的方式：

In [68]:
# 第一种，直接通过category关键字转换
df1 = pd.DataFrame({'A': np.arange(6), 'B': list('aabbca')})
df1['B'] = df1['B'].astype('category')
df1.B.dtypes

CategoricalDtype(categories=['a', 'b', 'c'], ordered=False)

In [69]:
# 第二种，直接指定为CategoricalDtype类型
from pandas.api.types import CategoricalDtype
df2 = pd.DataFrame({'A': np.arange(6), 'B': list('aabbca')})
df2['B'] = df2['B'].astype(CategoricalDtype(list('cba')))
df2.B.dtypes

CategoricalDtype(categories=['c', 'b', 'a'], ordered=False)

In [70]:
# 第三种，直接创建Categorical序列
cs = pd.Categorical(list('aabbca'), categories=list('cba'))
df3 = pd.DataFrame({'A': np.arange(6), 'B': cs})
df3.B.dtypes

CategoricalDtype(categories=['c', 'b', 'a'], ordered=False)

不管哪种方法，此时`B`列已经变成了`CategoricalDtype`类型，将其转为`index`就可以自动转为`CategoricalIndex`类型：

In [71]:
df4 = df2.set_index('B')
df4.index

CategoricalIndex(['a', 'a', 'b', 'b', 'c', 'a'], categories=['c', 'b', 'a'], ordered=False, name='B', dtype='category')

### `CategoricalIndex`索引类型的注意事项

1. 即使进行了选取，`CategoricalIndex`仍然会保存，注意`categories`属性，这样是为了计算更快：

In [75]:
df4.loc['a'].index
df4.groupby(level=0).sum().index

CategoricalIndex(['a', 'a', 'a'], categories=['c', 'b', 'a'], ordered=False, name='B', dtype='category')

CategoricalIndex(['c', 'b', 'a'], categories=['c', 'b', 'a'], ordered=False, name='B', dtype='category')

2. 对`CategoricalIndex`索引排序或者进行`groupby`计算，会按照类别的顺序排序，如下：

In [77]:
df4.sort_index().index
df4.groupby(level=0).sum().index

CategoricalIndex(['c', 'b', 'b', 'a', 'a', 'a'], categories=['c', 'b', 'a'], ordered=False, name='B', dtype='category')

CategoricalIndex(['c', 'b', 'a'], categories=['c', 'b', 'a'], ordered=False, name='B', dtype='category')

3. 对`CategoricalIndex`的重构和比较操作必须具有相同的类别，否则会引发类型错误`TypeError`。

In [87]:
df5 = df4.reindex(pd.Categorical(['a', 'c'], categories=list('abcde')))
df5.index
df4.index

CategoricalIndex(['a', 'a', 'a', 'c'], categories=['a', 'b', 'c', 'd', 'e'], ordered=False, name='B', dtype='category')

CategoricalIndex(['a', 'a', 'b', 'b', 'c', 'a'], categories=['c', 'b', 'a'], ordered=False, name='B', dtype='category')

In [90]:
try:
    pd.concat([df4, df5])
except Exception as e:
    print(e)

categories must match existing categories when appending


## `Float64Index`索引类型

浮点数的索引只要注意一点就是：不管是选取还是切片，`[]`和`loc`按值，`iloc`按位置：

In [101]:
indexf = pd.Index([1.5, 2, 3, 4.5, 5])
sf = pd.Series(range(5), index=indexf)
sf[2:3] # sf.loc[2:3]一样
sf.iloc[2:3]

2.0    1
3.0    2
dtype: int64

3.0    2
dtype: int64

注意：切片的时候，值不一定非要和标签的值一样，切片代表的是一个值的范围：

In [103]:
sf.loc[0:3.3]

1.5    0
2.0    1
3.0    2
dtype: int64

## `IntervalIndex`索引类型

### 手工创建`IntervalIndex`索引类型

我们可以手工创建`IntervalIndex`索引类型，如下：

In [4]:
df = pd.DataFrame(
    {
        'A': [1, 2, 3, 4]
    }, index=pd.IntervalIndex.from_breaks([0, 1, 2, 3, 4]))
df

,A
"(0, 1]",1
"(1, 2]",2
"(2, 3]",3
"(3, 4]",4


### `IntervalIndex`索引类型的选取

`IntervalInval`索引选取是沿着区间的右边界，要注意的是，如果标签的值在区间范围内，也可以进行选取，如下：

In [8]:
df.loc[2]
df.loc[[2, 3]]
df.loc[1.5]

A    2
Name: (1, 2], dtype: int64

,A
"(1, 2]",2
"(2, 3]",3


A    2
Name: (1, 2], dtype: int64

如果你就想根据区间的范围来选取，`pandas`在0.25版本以后，提供了`Interval`的顶层对象，要注意的是，`pd.Interval`需要准确的匹配，如果匹配不上，会抛出错误：

In [13]:
df.loc[pd.Interval(1, 2)]
try:
    df[pd.Interval(0.5, 1.5)]
except KeyError as e:
    print(f"KeyError: {e}")

A    2
Name: (1, 2], dtype: int64

KeyError: Interval(0.5, 1.5, closed='right')


你还可能会遇到这样的需求：你有一个区间，你需要把只要和这个区间有交集的所有数据筛选出来，可以使用`IntervalIndex`的`overlaps`方法：

In [15]:
idx = df.index.overlaps(pd.Interval(0.5, 2.5))
idx
df.loc[idx]

array([ True,  True,  True, False])

,A
"(0, 1]",1
"(1, 2]",2
"(2, 3]",3


### `cut`，`qcut`中的`IntervalIndex`

`cut()`和`qcut()`都返回一个分类对象（`Categorical`对象），如果未指定`labels`，则它们创建的`bins`在`.categories`属性中存储为一个`IntervalIndex`。

In [21]:
c = pd.cut(range(4), bins=2)
c
c.categories

[(-0.003, 1.5], (-0.003, 1.5], (1.5, 3.0], (1.5, 3.0]]
Categories (2, interval[float64]): [(-0.003, 1.5] < (1.5, 3.0]]

IntervalIndex([(-0.003, 1.5], (1.5, 3.0]],
              closed='right',
              dtype='interval[float64]')

`cut()`也接受它的bin参数的一个`IntervalIndex`，这里介绍一个`pandas`中常用的技巧：首先，调用`cut()`，将一些数据和`bins`设置为固定的数字，以生成区间（这里指生成的`Categorical`对象）对象。然后，将`.categories`的值作为随后调用`cut()`的`bin`参数传递，新数据就会被绑定到相同的区间。要注意的是：不在区间范围内的值会被设置为`NaN`。

In [22]:
pd.cut([0, 3, 5, 1], bins=c.categories)

[(-0.003, 1.5], (1.5, 3.0], NaN, (-0.003, 1.5]]
Categories (2, interval[float64]): [(-0.003, 1.5] < (1.5, 3.0]]

### `interval_range`方法

如果需要一个固定频率的区间，可以使用`interval_range()`函数来创建一个区间索引，`interval_range`的默认频率为1，表示数字间隔，用于时间则表示1天：

In [23]:
pd.interval_range(start=0, end=5)
pd.interval_range(start=pd.Timestamp('2017-01-01'), periods=4)
pd.interval_range(end=pd.Timedelta('3 days'), periods=3)

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]],
              closed='right',
              dtype='interval[int64]')

IntervalIndex([(2017-01-01, 2017-01-02], (2017-01-02, 2017-01-03], (2017-01-03, 2017-01-04], (2017-01-04, 2017-01-05]],
              closed='right',
              dtype='interval[datetime64[ns]]')

IntervalIndex([(0 days 00:00:00, 1 days 00:00:00], (1 days 00:00:00, 2 days 00:00:00], (2 days 00:00:00, 3 days 00:00:00]],
              closed='right',
              dtype='interval[timedelta64[ns]]')

`interval_range`有一个`freq`参数可以指定频率，如果用于时间类型的序列的话，则可以使用各种表示时间间隔的别名：

In [24]:
pd.interval_range(start=0, periods=5, freq=1.5)
pd.interval_range(start=pd.Timestamp('2017-01-01'), periods=4, freq='W')
pd.interval_range(start=pd.Timedelta('0 days'), periods=3, freq='9H')

IntervalIndex([(0.0, 1.5], (1.5, 3.0], (3.0, 4.5], (4.5, 6.0], (6.0, 7.5]],
              closed='right',
              dtype='interval[float64]')

IntervalIndex([(2017-01-01, 2017-01-08], (2017-01-08, 2017-01-15], (2017-01-15, 2017-01-22], (2017-01-22, 2017-01-29]],
              closed='right',
              dtype='interval[datetime64[ns]]')

IntervalIndex([(0 days 00:00:00, 0 days 09:00:00], (0 days 09:00:00, 0 days 18:00:00], (0 days 18:00:00, 1 days 03:00:00]],
              closed='right',
              dtype='interval[timedelta64[ns]]')

此外，`close`参数可用于指定区间在哪边关闭，除了`left`和`right`，还可以是`both`和`neighter`。默认情况下，间隔在右侧关闭。

In [25]:
pd.interval_range(start=0, end=4, closed='both')
pd.interval_range(start=0, end=4, closed='neither')

IntervalIndex([[0, 1], [1, 2], [2, 3], [3, 4]],
              closed='both',
              dtype='interval[int64]')

IntervalIndex([(0, 1), (1, 2), (2, 3), (3, 4)],
              closed='neither',
              dtype='interval[int64]')

## 关于索引的常见问题

### 整数索引的选取

整数索引很容易引起混淆以及产生一些微妙的bug，因此，`pandas`规定`loc`按照值进行选取（按值选取`pandas`给了一个专门的术语，叫label_based，即基于标签）。注意：按值选取，区间是右闭合的，`iloc`按照位置进行选取，另外，如果直接使用切片进行选取，则和`iloc`一样，按照位置进行选取。

In [35]:
df = pd.DataFrame(np.random.randn(5, 4))
# 按值选取，区间右闭合，因此选择0,1,2
df.loc[:2]

,0,1,2,3
0,-0.062441,-0.364633,-0.284526,2.110908
1,0.745154,1.058568,0.247652,-1.113749
2,-0.353624,0.913921,0.803328,-1.123403
3,-1.092782,0.232020,3.056465,0.030510
4,-0.024241,1.442851,-1.880936,0.619761


In [36]:
# 按位置选取，区间右界不闭合，因此选择0，1
df.iloc[:2]

,0,1,2,3
0,-0.062441,-0.364633,-0.284526,2.110908
1,0.745154,1.058568,0.247652,-1.113749


In [37]:
# 直接使用切片，则和iloc一样，按位置选取
df[:2]

,0,1,2,3
0,-0.062441,-0.364633,-0.284526,2.110908
1,0.745154,1.058568,0.247652,-1.113749


### 单调（升序或者降序）索引的选取

如果索引是单调递增或者递减的，那么基于标签的切片的边界可以在索引的范围之外，`pandas`还提供了`is_monotonic_increasing `和`is_monotonic_decreasing`2个属性来判断是否单调递增还是递减。

In [42]:
df = pd.DataFrame(index=[2, 3, 3, 4, 5], columns=['data'], data=list(range(5)))
df.index.is_monotonic_increasing
df.loc[0:3]
# 如果切片其实结束位置全部都在索引范围外，则返回空值
df.loc[6:10]

True

,data
2,0
3,1
3,2


,data


如果索引不是单调的，那么如果切片边界超出索引的范围，则会报错：

In [49]:
df = pd.DataFrame(
    index=[2, 3, 1, 4, 3, 5], columns=['data'], data=list(range(6)))
df.index.is_monotonic_increasing | df.index.is_monotonic_decreasing
try:
    df.loc[0:4]
except KeyError as e:
    print(f"KeyError: {e}")

False

KeyError: 0


不但切片边界要在索引范围内，而且要是唯一的（不唯一的显然会引起混淆，到底选取哪一个呢？）：

In [50]:
try:
    df.loc[2:3]
except KeyError as e:
    print(f"KeyError: {e}")

KeyError: 'Cannot get right slice bound for non-unique label: 3'


最后，`is_monotonic_increasing`和`is_monotonic_decreasing`只能检查弱单调（即单调升降，但是会有重复），如果要检测是否严格的单调，可以结合`is_unique`属性：

In [54]:
idx = pd.Index(['a', 'b', 'c', 'c'])
idx.is_monotonic_increasing
idx.is_monotonic_increasing & idx.is_unique

True

False

### 更改索引可能会改变`Series`的数据类型

索引的一些操作有可能会潜在的改变`Series`的数据类型（`dtype`）,先看下面这个例子：`NaN`在`pandas`（其实是`numpy`，`pandas`以`numpy`为基础）中，属于浮点类型，所以整个`Series`的`dtype`变为了浮点类型：

In [55]:
s1 = Series([1, 2, 3])
s1
s1.dtype
s2 = s1.reindex([0, 3])
s2
s2.dtype

0    1
1    2
2    3
dtype: int64

dtype('int64')

0    1.0
3    NaN
dtype: float64

dtype('float64')

再看下面的例子，又有浮点，又有布尔型的时候，被统一转换成了代表python对象的`object`类型：

In [56]:
s3 = pd.Series([True])
s3.dtype
res = s3.reindex_like(s1)
res.dtype
res

dtype('bool')

dtype('O')

0    True
1     NaN
2     NaN
dtype: object